In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# -*- coding: utf-8 -*-
"""
香港 KYC 合規系統
說明：
 - 註釋均為繁體中文（香港風格）
 - PDF 報告內容為英文（純段落，不含表格）
"""

# ------------------------------
# 安裝必要套件（Colab 執行時可用）
# ------------------------------
!pip install pandas openpyxl requests datasketch jellyfish fuzzywuzzy python-Levenshtein faker reportlab tqdm rapidfuzz -q

# ------------------------------
# 套件載入（標準導入）
# ------------------------------
import os, io, re, time, math, json, random, logging, warnings
from functools import lru_cache
from collections import defaultdict
from datetime import datetime, timedelta
from typing import List, Dict, Tuple, Optional

import numpy as np
import pandas as pd
import requests
from tqdm.notebook import tqdm

from fuzzywuzzy import fuzz
import jellyfish
from datasketch import MinHash, MinHashLSH

from reportlab.lib import colors
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle

# PDF 字型註冊（保持英文字型為內建字型）
from reportlab.pdfbase import pdfmetrics
# 不註冊外來中文字型以避免 OTF/Type1 問題（PDF 以英文輸出）

# Colab Drive 判斷與 mount（如果在非 Colab 環境也能執行但不 mount）
try:
    from google.colab import drive, files
    COLAB = True
except Exception:
    COLAB = False

# 日誌與警告設定
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
warnings.filterwarnings('ignore')

# ------------------------------
# 全域設定（可按需調整）
# ------------------------------
class Config:
    # 指定輸出目錄（依你的要求）
    OUTPUT_PATH = "/content/drive/MyDrive/Colab Notebooks/KYC"

    # 資料規模
    TOTAL_RECORDS = 15000         # （測試時可改小為 200 等）
    REAL_COMPANIES_RATIO = 0.10   # 10% 為知名公司族群

    # GLEIF 設定（查詢不受截斷）
    GLEIF_THRESHOLD = 75          # 使用相似度門檻（百分比）
    GLEIF_DELAY = 0.4             # 每次查詢延遲（秒）
    GLEIF_MAX_RETRIES = 2

    # OFAC 設定
    OFAC_SIMILARITY_THRESHOLD = 0.82
    OFAC_INJECT_RATE = 0.005      # 注入命中比例（測試用，0.005 => 0.5%）

    # 風險模型權重（可調）
    RISK_WEIGHTS = {
        'lei_missing': 15,
        'kyc_expired': 25,
        'sanctions_hit': 40,
        'high_risk_jurisdiction': 20,
        'large_transaction': 10
    }

    # 依註冊國家映射風險等級
    RISK_TIER_MAP = {
        'HK': ('中', 20),
        'VG': ('高', 50),
        'KY': ('高', 60),
        'CN': ('低', 10),
        'SG': ('中', 25),
        'DEFAULT': ('中', 30)
    }

    # 去重參數（降低閾值以更敏感）
    MINHASH_THRESHOLD = 0.60
    MINHASH_PERMUTATIONS = 64

    # 去重動態權重（風險導向）
    DEDUP_BASE_WEIGHTS = {'registration_match':0.45,'country_match':0.35,'name_similarity':0.20}
    DEDUP_RISK_PROFILE_WEIGHTS = {
        '極高風險': {'registration_match':0.70,'country_match':0.25,'name_similarity':0.05},
        '高':       {'registration_match':0.60,'country_match':0.30,'name_similarity':0.10},
        '中':       {'registration_match':0.45,'country_match':0.35,'name_similarity':0.20},
        '低':       {'registration_match':0.35,'country_match':0.45,'name_similarity':0.20},
        'DEFAULT':  {'registration_match':0.45,'country_match':0.35,'name_similarity':0.20}
    }
    DEDUP_REGNUM_MISSING_WEIGHTS = {'registration_match':0.0,'country_match':0.60,'name_similarity':0.40}
    DEDUP_REGNUM_INVALID_WEIGHTS = {'registration_match':0.20,'country_match':0.40,'name_similarity':0.40}
    DEDUP_SANCTIONS_BOOST = {'registration_match':0.80,'country_match':0.15,'name_similarity':0.05}

    # KYC 週期（銀行常見值）
    KYC_PERIODS_DAYS = {'極高風險':365,'高':730,'中':1095,'低':1825}

    # 檔案名稱設定
    FILE_RAW = "kyc_raw_data.csv"
    FILE_CLEAN = "kyc_cleaned_data.csv"
    FILE_LEI = "kyc_lei_enhanced_data.csv"
    FILE_DEDUP = "kyc_deduplicated_data.csv"
    FILE_DUP_TEST = "test_environment_duplicates.csv"
    FILE_EXCEL = "KYC合規報告_詳細版.xlsx"
    FILE_PDF = "KYC_Compliance_Summary_Report.pdf"
    FILE_GUIDE = "verification_guide.txt"
    FILE_MANUAL_REVIEW = "manual_review_notes.json"

    # 進度條開關
    PROGRESS_BAR_ENABLED = True

# 嘗試掛載 Google Drive（若在 Colab）
if COLAB:
    try:
        drive.mount('/content/drive', force_remount=True)
    except Exception as e:
        logging.warning(f"drive.mount 嘗試：{e}")
os.makedirs(Config.OUTPUT_PATH, exist_ok=True)
logging.info(f"輸出目錄：{Config.OUTPUT_PATH}")

# ------------------------------
# 品名清洗與 LEI 工具（KYCTools）
# ------------------------------
class KYCTools:
    # 常見公司後綴映射（英文與中文）
    SUFFIX_MAPPING = {
        'LTD':'LIMITED','LTD.':'LIMITED','LIMITED':'LIMITED',
        'INC':'INCORPORATED','INC.':'INCORPORATED','INCORPORATED':'INCORPORATED',
        'PLC':'PUBLIC LIMITED COMPANY','CO':'COMPANY','CO.':'COMPANY',
        'LLC':'LIMITED LIABILITY COMPANY','GROUP':'GROUP','HOLDINGS':'HOLDINGS',
        '有限公司':'LIMITED','集團':'GROUP','控股':'HOLDINGS'
    }

    # 常見特殊字元替換映射
    SPECIAL_CHAR_MAP = {'&':' AND ','@':' AT ','/':' ','\\':' ','，':',','。':'.','（':'(','）':')'}

    @staticmethod
    def fullwidth_to_halfwidth(s: str) -> str:
        """全形轉半形（為了處理中英混合輸入）"""
        result = []
        for ch in s:
            code = ord(ch)
            if 0xFF01 <= code <= 0xFF5E:
                code -= 0xFEE0
            result.append(chr(code))
        return ''.join(result)

    @staticmethod
    def normalize_whitespace(s: str) -> str:
        """將多個空白收斂為單一空白並 trim"""
        return re.sub(r'\s+', ' ', s).strip()

    @staticmethod
    def clean_company_name(name: str) -> str:
        """
        清理與標準化公司名稱：
         - 全形轉半形、替換特殊字元
         - 移除不必要符號、保留中文與英文
         - 標準化常見後綴並移除
         - 回傳大寫（中文保留）
        """
        if pd.isna(name) or str(name).strip() == "":
            return "MISSING_NAME"
        s = str(name).strip()
        s = KYCTools.fullwidth_to_halfwidth(s)
        for k,v in KYCTools.SPECIAL_CHAR_MAP.items():
            s = s.replace(k, v)
        s = re.sub(r'[\[\]\{\}\|<>~`©®™\*]', ' ', s)
        s = re.sub(r"[^A-Za-z0-9\u4e00-\u9fff\s\-/()]", " ", s)
        s = KYCTools.normalize_whitespace(s)
        s = s.upper()
        if '/' in s:
            parts = [p.strip() for p in s.split('/') if p.strip()]
            english_candidate = ''
            for p in parts:
                if re.search(r'[A-Z]', p):
                    english_candidate = english_candidate + (' ' + p if english_candidate else p)
            s = english_candidate if english_candidate else parts[0]
        words = s.split()
        cleaned = []
        for w in words:
            w2 = w.strip().strip('.').upper()
            cleaned.append(KYCTools.SUFFIX_MAPPING.get(w2, w2))
        s = ' '.join(cleaned)
        s = KYCTools.normalize_whitespace(s)
        tokens = s.split()
        filtered = [t for t in tokens if t not in set(KYCTools.SUFFIX_MAPPING.values())]
        if filtered:
            s = ' '.join(filtered)
        s = KYCTools.normalize_whitespace(s)
        return s if s else "MISSING_NAME"

    @staticmethod
    @lru_cache(maxsize=20000)
    def calculate_similarity(name1: str, name2: str) -> float:
        """
        計算兩個名稱相似度：
         - Jaro-Winkler 與 token_sort_ratio 混合
         - 回傳 0~1 浮點
        """
        if not name1 or not name2:
            return 0.0
        try:
            jw = jellyfish.jaro_winkler_similarity(name1, name2)
        except Exception:
            jw = 0.0
        token = fuzz.token_sort_ratio(name1, name2) / 100.0
        return round(0.7 * jw + 0.3 * token, 4)

    @staticmethod
    def lei_iso17442_check(lei: Optional[str]) -> bool:
        """
        嚴格的 ISO 17442 LEI 校驗（包含檢查碼計算）
        """
        if not lei or pd.isna(lei):
            return False
        lei = str(lei).strip().upper()
        if len(lei) != 20:
            return False
        body = lei[:18]; check = lei[18:]
        numeric_parts = []
        for ch in body:
            if ch.isdigit():
                numeric_parts.append(ch)
            elif 'A' <= ch <= 'Z':
                numeric_parts.append(str(ord(ch) - 55))
            else:
                return False
        numeric_str = ''.join(numeric_parts) + "00"
        total = 0
        for i in range(0, len(numeric_str), 9):
            part = numeric_str[i:i+9]
            total = (total * (10**len(part)) + int(part)) % 97
        computed = (98 - total) % 97
        try:
            check_int = int(check)
        except:
            return False
        return computed == check_int

    @staticmethod
    def safe_lei_validation(lei: Optional[str]) -> bool:
        """寬鬆入口後接嚴格校驗"""
        if not lei or pd.isna(lei):
            return False
        lei = str(lei).strip().upper()
        if not re.match(r'^[0-9A-Z]{20}$', lei):
            return False
        return KYCTools.lei_iso17442_check(lei)

    @staticmethod
    def generate_registration_number(country: str) -> str:
        """根據國家生成一個類似的登記號"""
        if country == 'HK':
            return str(random.randint(1000000, 19999999))
        elif country == 'VG':
            return f"VG{random.randint(100000, 999999)}"
        elif country == 'KY':
            return f"KY{random.randint(100000, 999999)}"
        elif country == 'SG':
            return f"{random.randint(2000,2024)}{random.randint(10000,99999)}"
        else:
            return f"{country}-{random.randint(1000,9999)}"

    @staticmethod
    def generate_valid_lei(prefix: Optional[str] = None) -> Optional[str]:
        """
        自行生成一個合法格式且通過檢查碼的 LEI
        - prefix: 可指定前 6 位（例如 529900）
        - 回傳 20 字元 LEI 或 None（理論上會生成成功）
        """
        if prefix and not re.match(r'^[0-9]{6}$', str(prefix)):
            prefix = None
        if not prefix:
            prefix = random.choice(['529900','254900','969500','213800','377900'])
        rest = ''.join(random.choices('0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ', k=12))
        body = prefix + rest  # 18 chars
        numeric_parts = []
        for ch in body:
            if ch.isdigit():
                numeric_parts.append(ch)
            elif 'A' <= ch <= 'Z':
                numeric_parts.append(str(ord(ch) - 55))
            else:
                return None
        numeric_str = ''.join(numeric_parts) + "00"
        total = 0
        for i in range(0, len(numeric_str), 9):
            part = numeric_str[i:i+9]
            total = (total * (10**len(part)) + int(part)) % 97
        computed = (98 - total) % 97
        check = f"{computed:02d}"
        lei = body + check
        if KYCTools.safe_lei_validation(lei):
            return lei
        # fallback loop
        for _ in range(10):
            rest = ''.join(random.choices('0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ', k=12))
            body = prefix + rest
            numeric_parts = []
            for ch in body:
                numeric_parts.append(ch if ch.isdigit() else str(ord(ch)-55))
            numeric_str = ''.join(numeric_parts) + "00"
            total = 0
            for i in range(0, len(numeric_str), 9):
                part = numeric_str[i:i+9]
                total = (total * (10**len(part)) + int(part)) % 97
            computed = (98 - total) % 97
            lei = body + f"{computed:02d}"
            if KYCTools.safe_lei_validation(lei):
                return lei
        return None

# ------------------------------
# 載入 OFAC SDN 名單範例（加速測試、並儲存本地副本）
# ------------------------------
def load_sdn_sample(limit=500):
    url = "https://www.treasury.gov/ofac/downloads/sdn.csv"
    try:
        r = requests.get(url, timeout=25)
        text = r.text
        lines = [l for l in text.splitlines() if l and not l.startswith('#')]
        df = pd.read_csv(io.StringIO('\n'.join(lines)), header=None, dtype=str, low_memory=False)
        names = []
        if df.shape[1] >= 2:
            names += df[1].dropna().astype(str).tolist()
        names = [n.strip() for n in names if isinstance(n, str) and n.strip()]
        names = list(dict.fromkeys(names))
        return names[:limit]
    except Exception as e:
        logging.warning(f"下載 SDN 失敗或網路不可用：{e}")
        return ["CENTRAL BANK OF IRAN","VLADIMIR PUTIN","NORTH KOREA","SYRIAN ARAB REPUBLIC"]

SDN_SAMPLE = load_sdn_sample(limit=500)

# ------------------------------
# STEP 1：產生測試資料（含 2–5% 重複、OFAC 注入、LEI 生成）
# ------------------------------
def generate_realistic_dataset(total: int = Config.TOTAL_RECORDS,
                               duplicate_rate_min: float = 0.02,
                               duplicate_rate_max: float = 0.05,
                               ofac_inject_rate: float = Config.OFAC_INJECT_RATE,
                               lei_real_rate: float = 0.75,   # 真實公司中生成 LEI 的機率
                               lei_other_rate: float = 0.05): # 其他公司生成 LEI 的機率
    """
    生成模擬 KYC 資料：
    - 會注入 OFAC 名稱到一部分紀錄（以測試 OFAC 命中）
    - 會生成合理分布的 last_kyc_review（偏向近期）
    - 會為真實公司提供高機率的 LEI（可檢驗覆蓋率）
    - 會在最後製造 2-5% 的近似重複紀錄
    """
    from faker import Faker
    Faker.seed(2025); random.seed(2025); np.random.seed(2025)
    fake_en = Faker('en_US')

    data = []
    real_count = int(total * Config.REAL_COMPANIES_RATIO)
    REAL_COMPANY_PAIRS = [
        ("HSBC HOLDINGS PLC","滙豐控股有限公司"),
        ("STANDARD CHARTERED PLC","渣打集團有限公司"),
        ("BANK OF CHINA LIMITED","中國銀行股份有限公司"),
        ("AIA GROUP LIMITED","友邦保險控股有限公司"),
        ("HANG SENG BANK LIMITED","恒生銀行有限公司"),
        ("TENCENT HOLDINGS LIMITED","騰訊控股有限公司"),
        ("ALIBABA GROUP HOLDING LIMITED","阿里巴巴集團控股有限公司"),
        ("PING AN INSURANCE GROUP","中國平安保險集團")
    ]

    countries = ['HK','VG','KY','CN','SG']
    weights = [0.5,0.2,0.15,0.1,0.05]

    # 預選 OFAC 注入索引
    num_ofac = int(total * ofac_inject_rate)
    ofac_indices = set(random.sample(range(total), k=num_ofac)) if num_ofac > 0 else set()

    for i in range(total):
        is_real = (i < real_count)
        # 若為注入索引，直接用 SDN 範例名稱以確保命中
        if i in ofac_indices:
            sdn_name = SDN_SAMPLE[random.randint(0, len(SDN_SAMPLE)-1)]
            company_name = sdn_name
        else:
            if is_real:
                en, zh = random.choice(REAL_COMPANY_PAIRS)
                company_name = f"{en} / {zh}"
            else:
                base_en = fake_en.company().replace(',', '').replace('.', '')
                suffix = random.choice([" Limited"," Ltd"," Group"," Holdings"," PLC"])
                en_name = f"{base_en}{suffix}"
                first_word = base_en.split()[0] if base_en.split() else ""
                translit_map = {'A':'亞','B':'百','C':'中','D':'大','E':'易','F':'富','G':'金','H':'亨','I':'愛','J':'捷','K':'國'}
                zh_core = translit_map.get(first_word[0].upper(), '嘉') if first_word else '嘉'
                industry = random.choice(["投資","地產","金融","科技","貿易","實業"])
                company_name = f"{en_name} / {zh_core}{industry}有限公司"

        country = random.choices(countries, weights=weights, k=1)[0]
        reg_no = KYCTools.generate_registration_number(country)
        aum = round(min(np.random.lognormal(3.5, 1.5), 10000), 2)
        inc_date = fake_en.date_between(start_date=datetime(1990,1,1), end_date=datetime(2024,12,31))

        # last_kyc_review：指數分布偏向近期
        days_ago = int(min(np.random.exponential(scale=400), 2500))
        last_review_date = datetime.today().date() - timedelta(days=days_ago)

        # LEI 生成（提高真實公司覆蓋率）
        lei = None
        if is_real:
            if random.random() < lei_real_rate:
                lei = KYCTools.generate_valid_lei()
        else:
            if random.random() < lei_other_rate:
                lei = KYCTools.generate_valid_lei()

        rec = {
            'record_id': f"KYC{str(i+1).zfill(6)}",
            'company_name': company_name,
            'registered_country': country,
            'reg_no': reg_no,
            'lei': lei,
            'aum_usd_millions': aum,
            'incorporation_date': pd.to_datetime(inc_date),
            'last_kyc_review': pd.to_datetime(last_review_date),
            'is_real_company': is_real,
            'data_source': 'generated'
        }
        data.append(rec)

    df = pd.DataFrame(data)

    # 製造 2-5% 的近似重複（加上小幅噪音）
    dup_rate = random.uniform(duplicate_rate_min, duplicate_rate_max)
    num_dup = int(total * dup_rate)
    if num_dup > 0:
        dup_indices = random.sample(list(df.index), k=num_dup)
        duplicates = []
        for orig_idx in dup_indices:
            orig = df.loc[orig_idx].to_dict()
            cname = orig['company_name']
            if '/' in cname:
                left, right = [p.strip() for p in cname.split('/')[:2]]
                if random.random() < 0.5:
                    left = left + " "
                else:
                    left = left.lower()
                new_name = f"{left} / {right}"
            else:
                new_name = cname + " "
            rno = orig['reg_no']
            rno_new = rno
            if isinstance(rno, str) and re.search(r'\d', rno):
                last_digits = re.findall(r'\d+', rno)
                if last_digits:
                    last = last_digits[-1]
                    try:
                        new_last = str((int(last[-1]) + 1) % 10)
                        rno_new = re.sub(last+'$', last[:-1]+new_last if len(last)>1 else new_last, rno)
                    except:
                        rno_new = rno
            lr = orig['last_kyc_review']
            if pd.isna(lr):
                lr_new = lr
            else:
                lr_new = pd.to_datetime(lr) + pd.to_timedelta(random.randint(-30,30), unit='d')
            new_rec = orig.copy()
            new_rec['record_id'] = f"KYC_DUP_{orig_idx}"
            new_rec['company_name'] = new_name
            new_rec['reg_no'] = rno_new
            new_rec['last_kyc_review'] = lr_new
            duplicates.append(new_rec)
        if duplicates:
            df = pd.concat([df, pd.DataFrame(duplicates)], ignore_index=True).reset_index(drop=True)

    # 儲存原始資料
    raw_path = os.path.join(Config.OUTPUT_PATH, Config.FILE_RAW)
    df.to_csv(raw_path, index=False, encoding='utf-8-sig')
    logging.info(f"Generated {len(df)} records (duplicates: {num_dup}, OFAC-injected: {len(ofac_indices)}) -> saved to {raw_path}")
    return df

# 產生資料（執行此行會耗費時間，測試請先把 Config.TOTAL_RECORDS 改小）
raw_df = generate_realistic_dataset()

# ------------------------------
# STEP 2：清洗與標準化
# ------------------------------
def clean_and_standardize(df: pd.DataFrame) -> pd.DataFrame:
    """
    清洗步驟：
     - company_name 清理
     - 拆英中（若有）
     - reg_no 正規化
     - 風險 tier 與基礎分數
     - 判定 KYC 是否到期（依 Config.KYC_PERIODS_DAYS）
    """
    df2 = df.copy()
    df2['company_name_clean'] = df2['company_name'].apply(KYCTools.clean_company_name)

    def split_bilingual(s):
        if pd.isna(s): return ("","")
        if '/' in s:
            parts = [p.strip() for p in s.split('/') if p.strip()]
            en = parts[0] if re.search(r'[A-Za-z]', parts[0]) else ''
            zh = parts[1] if len(parts) > 1 else ''
            return en, zh
        en = re.sub(r'[\u4e00-\u9fff]+', ' ', str(s)).strip()
        zh = re.sub(r'[A-Za-z0-9\W_]+', ' ', str(s)).strip()
        return en, zh

    df2[['company_name_en','company_name_zh']] = df2['company_name'].apply(lambda x: pd.Series(split_bilingual(x)))
    df2['reg_no_normalized'] = df2['reg_no'].astype(str).str.upper().str.replace(r'[\s\-/]', '', regex=True)
    df2['risk_tier'] = df2['registered_country'].apply(lambda x: Config.RISK_TIER_MAP.get(x, Config.RISK_TIER_MAP['DEFAULT'])[0])
    df2['base_risk_score'] = df2['registered_country'].apply(lambda x: Config.RISK_TIER_MAP.get(x, Config.RISK_TIER_MAP['DEFAULT'])[1])

    today = datetime.today().date()
    def kyc_due(row):
        last = row['last_kyc_review']
        try:
            lastd = pd.to_datetime(last).date()
        except:
            return True
        days_allowed = Config.KYC_PERIODS_DAYS.get(row['risk_tier'], 1095)
        return (today - lastd).days > days_allowed

    df2['kyc_refresh_due'] = df2.apply(kyc_due, axis=1)

    def days_to_next(row):
        try:
            lastd = pd.to_datetime(row['last_kyc_review']).date()
        except:
            return 0
        days_allowed = Config.KYC_PERIODS_DAYS.get(row['risk_tier'], 1095)
        rem = days_allowed - (today - lastd).days
        return max(0, rem)

    df2['days_until_next_review'] = df2.apply(days_to_next, axis=1)
    df2['large_transaction_flag'] = df2['aum_usd_millions'] > 500
    df2['lei_valid'] = df2['lei'].apply(lambda x: KYCTools.safe_lei_validation(x) if pd.notna(x) else False)

    clean_path = os.path.join(Config.OUTPUT_PATH, Config.FILE_CLEAN)
    df2.to_csv(clean_path, index=False, encoding='utf-8-sig')
    logging.info(f"Cleaned data saved: {clean_path}")
    return df2

cleaned_df = clean_and_standardize(raw_df)

# ------------------------------
# STEP 3：GLEIF LEI 增強（不限制查詢數量，會嘗試對每個 candidate 查詢）
# ------------------------------
class GLEIFClient:
    """簡易 GLEIF 客戶端；具備重試與簡化查詢邏輯"""
    def __init__(self, base_url="https://api.gleif.org/api/v1/lei-records"):
        self.base_url = base_url
        self.session = requests.Session()
        self.session.headers.update({'User-Agent':'KYC-System-v16','Accept':'application/vnd.api+json'})
        self.cache = {}

    def fetch_lei(self, name: str, max_retries:int = Config.GLEIF_MAX_RETRIES) -> Optional[str]:
        key = name.strip().upper()
        if not key:
            return None
        if key in self.cache:
            return self.cache[key]
        for attempt in range(max_retries+1):
            try:
                params = {"filter[entity.legalName]": name, "page[size]":1}
                resp = self.session.get(self.base_url, params=params, timeout=15)
                if resp.status_code == 200:
                    j = resp.json()
                    if j.get('data'):
                        lei = j['data'][0].get('attributes', {}).get('lei')
                        if lei and KYCTools.safe_lei_validation(lei):
                            self.cache[key] = lei
                            return lei
                # 嘗試簡化名稱（去掉常見 legal suffix）
                if attempt == 0:
                    simplified = re.sub(r'\b(LIMITED|LTD|PLC|INC|INCORPORATED|CORPORATION|GROUP|HOLDINGS)\b','', name, flags=re.IGNORECASE).strip()
                    if simplified and simplified != name:
                        params["filter[entity.legalName]"] = simplified
                        resp = self.session.get(self.base_url, params=params, timeout=10)
                        if resp.status_code == 200:
                            j = resp.json()
                            if j.get('data'):
                                lei = j['data'][0].get('attributes',{}).get('lei')
                                if lei and KYCTools.safe_lei_validation(lei):
                                    self.cache[key] = lei
                                    return lei
            except Exception as e:
                logging.warning(f"GLEIF attempt {attempt} error: {e}")
                time.sleep(1 + attempt)
        self.cache[key] = None
        return None

    def batch_fetch_lei(self, names: List[str]) -> Dict[str,str]:
        out = {}
        iterator = tqdm(names, desc="GLEIF 查詢", leave=False) if Config.PROGRESS_BAR_ENABLED else names
        for n in iterator:
            if not n or len(n) < 3:
                continue
            key = n.strip().upper()
            if key in self.cache and self.cache[key] is not None:
                out[n] = self.cache[key]
                continue
            lei = self.fetch_lei(n)
            if lei:
                out[n] = lei
            time.sleep(Config.GLEIF_DELAY)
        return out

def enhance_lei_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    對所有 candidate 執行 GLEIF 查詢（不再限制數量）
    優先保留原始 lei（若存在且驗證通過），否則填充查詢到的 lei
    """
    logging.info("開始 LEI 增強（嘗試大量 GLEIF 查詢）")
    client = GLEIFClient()
    candidates = df['company_name_clean'].dropna().unique().tolist()
    candidates = [c for c in candidates if len(c) > 3]
    try:
        lei_map = client.batch_fetch_lei(candidates)
    except Exception as e:
        logging.warning(f"GLEIF batch 查詢失敗：{e}")
        lei_map = {}
    df2 = df.copy()
    def upd(row):
        if pd.notna(row.get('lei')) and KYCTools.safe_lei_validation(row.get('lei')):
            return row.get('lei')
        name = row.get('company_name_clean','')
        if name in lei_map and lei_map[name]:
            return lei_map[name]
        # 近似比對
        best = None; bests = 0.0
        for k,v in lei_map.items():
            s = KYCTools.calculate_similarity(name, k)
            if s > bests and s >= (Config.GLEIF_THRESHOLD / 100.0):
                bests = s; best = v
        return best
    df2['lei_enhanced'] = df2.apply(upd, axis=1)
    df2['lei_enhanced_valid'] = df2['lei_enhanced'].apply(lambda x: KYCTools.safe_lei_validation(x) if pd.notna(x) else False)
    lei_path = os.path.join(Config.OUTPUT_PATH, Config.FILE_LEI)
    df2.to_csv(lei_path, index=False, encoding='utf-8-sig')
    logging.info(f"LEI 增強結果已儲存：{lei_path}")
    return df2

enhanced_df = enhance_lei_data(cleaned_df)

# ------------------------------
# STEP 4：OFAC 篩查（解析本地或線上 SDN，並計算相似度）
# ------------------------------
class OFACScreener:
    """
    OFAC 篩查器：
     - 若有本地 sdn.csv 則使用，否則嘗試下載並本地儲存
     - 會建立簡單字詞索引以加速比對
    """
    def __init__(self):
        local = os.path.join(Config.OUTPUT_PATH, "sdn.csv")
        lines = []
        if os.path.exists(local):
            logging.info(f"使用本地 SDN：{local}")
            with open(local, 'r', encoding='utf-8', errors='ignore') as f:
                lines = [l for l in f.read().splitlines() if l and not l.startswith('#')]
        else:
            try:
                logging.info("下載 OFAC SDN 列表（並儲存本地副本）...")
                r = requests.get("https://www.treasury.gov/ofac/downloads/sdn.csv", timeout=30)
                text = r.text
                lines = [l for l in text.splitlines() if l and not l.startswith('#')]
                with open(local, 'w', encoding='utf-8') as f:
                    f.write('\n'.join(lines))
            except Exception as e:
                logging.warning(f"OFAC 下載失敗：{e}")
                lines = []

        self.entities = self.parse_sdn_lines(lines)
        self.index = self.build_index(self.entities)

    def parse_sdn_lines(self, lines: List[str]) -> List[str]:
        entities = []
        if not lines:
            return ["CENTRAL BANK OF IRAN","VLADIMIR PUTIN","NORTH KOREA","SYRIAN ARAB REPUBLIC"]
        try:
            df = pd.read_csv(io.StringIO('\n'.join(lines)), header=None, dtype=str, low_memory=False)
            if df.shape[1] >= 2:
                entities += df[1].dropna().astype(str).tolist()
            if df.shape[1] >= 12:
                remarks = df[11].dropna().astype(str).tolist()
                for r in remarks:
                    m = re.findall(r'["\']([^"\']+)["\']', r)
                    if m:
                        entities.extend(m)
                    else:
                        parts = re.split(r'[;,]', r)
                        for p in parts:
                            p = p.strip()
                            if len(p) > 2:
                                entities.append(p)
            ents = list({e.strip().upper() for e in entities if isinstance(e, str) and e.strip()})
            logging.info(f"解析 OFAC 實體數量：{len(ents)}")
            return ents
        except Exception as e:
            logging.warning(f"解析 SDN CSV 失敗：{e}")
            return list({l.strip().upper() for l in lines if l and l.strip()})

    def build_index(self, names: List[str]) -> Dict[str,List[str]]:
        idx = defaultdict(list)
        for n in names:
            for w in set(n.split()):
                if len(w) >= 4:
                    idx[w].append(n)
        return idx

    def screen_company(self, company_name: str) -> Tuple[bool, float, str]:
        """
        對單一公司名稱執行 OFAC 比對，回傳 (hit:bool, confidence:float, matched_entity:str)
        """
        if not company_name or company_name == "MISSING_NAME":
            return False, 0.0, ""
        s = company_name.upper()
        trans_map = {'IRAN':['伊朗','依朗'],'PUTIN':['普京'],'NORTH KOREA':['朝鮮','北韓']}
        for k, variants in trans_map.items():
            for v in variants:
                if v.upper() in s or s in v.upper():
                    return True, 0.95, k
                if KYCTools.calculate_similarity(s, v.upper()) >= 0.85:
                    return True, KYCTools.calculate_similarity(s, v.upper()), k
        cand = set()
        for w in set(s.split()):
            if len(w) >= 4 and w in self.index:
                cand.update(self.index[w])
        if not cand:
            cand = set(self.entities)
        best_s = 0.0; best_ent = ""
        for ent in cand:
            score = KYCTools.calculate_similarity(s, ent.upper())
            if score > best_s:
                best_s = score; best_ent = ent
        hit = best_s >= Config.OFAC_SIMILARITY_THRESHOLD
        confidence = min(0.99, best_s * 1.1) if hit else max(0.01, best_s * 0.8)
        return bool(hit), round(confidence, 3), (best_ent if hit else "")

def perform_ofac_screening(df: pd.DataFrame) -> pd.DataFrame:
    logging.info("執行 OFAC 篩查...")
    screener = OFACScreener()
    rows = []
    pbar = tqdm(total=len(df), desc="OFAC 篩查") if Config.PROGRESS_BAR_ENABLED else None
    for idx, row in df.iterrows():
        hit, conf, matched = screener.screen_company(row['company_name_clean'])
        rows.append({'ofac_hit': hit, 'ofac_confidence': conf, 'ofac_matched_entity': matched if hit else ""})
        if pbar: pbar.update(1)
    if pbar: pbar.close()
    ofac_df = pd.DataFrame(rows, index=df.index)
    out = pd.concat([df, ofac_df], axis=1)
    logging.info(f"OFAC 篩查完成。命中數：{int(out['ofac_hit'].sum())}")
    return out

screened_df = perform_ofac_screening(enhanced_df)

# ------------------------------
# STEP 5：風險評估
# ------------------------------
class RiskAssessor:
    """計算風險因子與合成分數"""
    def __init__(self):
        self.weights = Config.RISK_WEIGHTS

    def calculate_factors(self, row):
        base = row.get('base_risk_score', 30) or 30
        lei_risk = self.weights['lei_missing'] if not row.get('lei_enhanced_valid', False) else 0
        kyc_risk = self.weights['kyc_expired'] if row.get('kyc_refresh_due', False) else 0
        sanctions_risk = self.weights['sanctions_hit'] if row.get('ofac_hit', False) else 0
        jurisdiction_risk = self.weights['high_risk_jurisdiction'] if row.get('risk_tier') in ['高','極高風險'] else 0
        transaction_risk = self.weights['large_transaction'] if row.get('large_transaction_flag', False) else 0
        confidence_penalty = row.get('ofac_confidence', 0) * 10 if row.get('ofac_hit', False) else 0
        return {'base_risk': base, 'lei_risk': lei_risk, 'kyc_risk': kyc_risk, 'sanctions_risk': sanctions_risk, 'jurisdiction_risk': jurisdiction_risk, 'transaction_risk': transaction_risk, 'confidence_penalty': confidence_penalty}

    def composite_score(self, factors):
        total = sum(factors.values())
        try:
            score = 100 * (1 - 1/(1 + math.exp(0.08*(total - 50))))
        except:
            score = min(100, total)
        return round(max(0, min(100, score)), 1)

    def category(self, score):
        if score >= 80: return '極高風險', '🔴'
        if score >= 60: return '高', '🟠'
        if score >= 40: return '中', '🟡'
        return '低', '🟢'

    def assess(self, row):
        f = self.calculate_factors(row)
        s = self.composite_score(f)
        cat, emoji = self.category(s)
        primary = max(f.items(), key=lambda x: x[1])[0] if any(f.values()) else 'base_risk'
        return {'risk_score': s, 'risk_category': cat, 'risk_emoji': emoji, 'risk_factors': f, 'primary_risk_factor': primary}

def perform_risk_assessment(df: pd.DataFrame) -> pd.DataFrame:
    logging.info("執行風險評估...")
    assessor = RiskAssessor()
    rows = []
    pbar = tqdm(total=len(df), desc="風險評估") if Config.PROGRESS_BAR_ENABLED else None
    for idx, r in df.iterrows():
        rows.append(assessor.assess(r))
        if pbar: pbar.update(1)
    if pbar: pbar.close()
    riskdf = pd.DataFrame(rows, index=df.index)
    out = pd.concat([df, riskdf], axis=1)
    logging.info("風險評估完成。")
    return out

assessed_df = perform_risk_assessment(screened_df)

# ------------------------------
# STEP 6：智能去重（動態權重、MinHash + LSH）
# ------------------------------
class DeduplicationEngine:
    """動態權重去重引擎：先做完全重複移除，再用 MinHashLSH 找模糊重複"""
    def __init__(self, threshold=Config.MINHASH_THRESHOLD, num_perm=Config.MINHASH_PERMUTATIONS):
        self.threshold = threshold
        self.num_perm = num_perm
        self.lsh = MinHashLSH(threshold=threshold, num_perm=num_perm)
        self.minhash_map = {}

    def create_minhash(self, text: str):
        m = MinHash(num_perm=self.num_perm)
        tokens = text.split()
        for t in tokens:
            m.update(t.encode('utf-8'))
            if len(t) >= 3:
                for i in range(len(t)-2):
                    m.update(t[i:i+3].encode('utf-8'))
        return m

    def compute_dynamic_weights(self, row):
        rc = row.get('risk_category') or row.get('risk_tier') or 'DEFAULT'
        risk_key = rc if rc in Config.DEDUP_RISK_PROFILE_WEIGHTS else 'DEFAULT'
        weights = Config.DEDUP_RISK_PROFILE_WEIGHTS.get(risk_key, Config.DEDUP_BASE_WEIGHTS).copy()
        reg = (row.get('reg_no_normalized') or '').strip()
        reg_exists = bool(reg and reg.upper() not in ['NONE','NAN',''])
        if not reg_exists:
            if risk_key == '低':
                weights = Config.DEDUP_REGNUM_MISSING_WEIGHTS.copy()
            else:
                return None
        else:
            if not re.match(r'^[A-Z0-9\-]{3,}$', reg):
                weights = Config.DEDUP_REGNUM_INVALID_WEIGHTS.copy()
        if row.get('ofac_hit', False):
            weights = Config.DEDUP_SANCTIONS_BOOST.copy()
        s = sum(weights.values())
        if s > 0:
            for k in weights:
                weights[k] = weights[k] / s
        return weights

    def calculate_similarity(self, r1, r2):
        reg1 = (r1.get('reg_no_normalized') or '').upper()
        reg2 = (r2.get('reg_no_normalized') or '').upper()
        reg_match = 1.0 if (reg1 and reg2 and reg1 == reg2) else 0.0
        country_match = 1.0 if (r1.get('registered_country') == r2.get('registered_country')) else 0.0
        name_sim = KYCTools.calculate_similarity(r1.get('company_name_clean',''), r2.get('company_name_clean',''))
        w1 = self.compute_dynamic_weights(r1); w2 = self.compute_dynamic_weights(r2)
        if w1 is None or w2 is None:
            w = {'registration_match':0.0,'country_match':0.6,'name_similarity':0.4}
        else:
            w = {k:(w1.get(k,0) + w2.get(k,0))/2.0 for k in ['registration_match','country_match','name_similarity']}
        combined = reg_match * w['registration_match'] + country_match * w['country_match'] + name_sim * w['name_similarity']
        return combined

    def find_duplicates(self, df: pd.DataFrame, key_columns: List[str]):
        # Step1: 完全相同鍵值移除（exact dedup）
        df2 = df.copy()
        exact_mask = df2.duplicated(subset=key_columns, keep='first')
        removed_exact = exact_mask.sum()
        df_exact = df2[~exact_mask].copy()

        # Step2: 建立 LSH 並搜尋相似群組
        self.lsh = MinHashLSH(threshold=self.threshold, num_perm=self.num_perm)
        self.minhash_map = {}
        index_map = {}
        for idx, row in df_exact.iterrows():
            combined = f"{row.get('company_name_clean','')} {row.get('registered_country','')} {row.get('reg_no_normalized','')}"
            mh = self.create_minhash(combined)
            sid = str(idx)
            try:
                self.lsh.insert(sid, mh)
            except Exception:
                pass
            self.minhash_map[sid] = mh
            index_map[sid] = idx

        processed = set()
        groups = []
        pbar = tqdm(total=len(df_exact), desc="查找相似記錄") if Config.PROGRESS_BAR_ENABLED else None
        for sid, orig_idx in index_map.items():
            if orig_idx in processed:
                if pbar: pbar.update(1)
                continue
            mh = self.minhash_map[sid]
            try: self.lsh.remove(sid)
            except: pass
            candidate_keys = self.lsh.query(mh)
            try: self.lsh.insert(sid, mh)
            except: pass
            candidate_idxs = [index_map[k] for k in candidate_keys if k in index_map and index_map[k] != orig_idx]
            valid_similar = []
            for cidx in candidate_idxs:
                sim_score = self.calculate_similarity(df_exact.loc[orig_idx], df_exact.loc[cidx])
                if sim_score >= self.threshold:
                    valid_similar.append((cidx, sim_score))
            if valid_similar:
                group = [orig_idx] + [g[0] for g in valid_similar]
                groups.append({'group': group, 'scores': [g[1] for g in valid_similar]})
                processed.update(group)
            processed.add(orig_idx)
            if pbar: pbar.update(1)
        if pbar: pbar.close()

        # Step3: 決定保留哪筆（以 OFAC 命中或最高 risk_score 保留）
        to_remove = set(); manual_review = []
        for g in groups:
            group = g['group']
            if len(group) <= 1:
                continue
            sub = df_exact.loc[group]
            sanc = sub[sub['ofac_hit'] == True] if 'ofac_hit' in sub.columns else pd.DataFrame()
            if not sanc.empty:
                keep_idx = sanc['risk_score'].idxmax()
            else:
                keep_idx = sub['risk_score'].idxmax()
            for rid in group:
                if rid != keep_idx:
                    to_remove.add(rid)
            if max(g['scores']) > 0.98:
                manual_review.append({'primary': keep_idx, 'duplicates': [r for r in group if r != keep_idx], 'max_sim': max(g['scores'])})

        df_final = df_exact.drop(index=list(to_remove)).reset_index(drop=True)

        # 匯出測試去重清單（若有）
        dup_test_path = os.path.join(Config.OUTPUT_PATH, Config.FILE_DUP_TEST)
        test_rows = []
        for item in groups:
            grp = item['group']
            sub = df_exact.loc[grp]
            keep_idx = sub['risk_score'].idxmax()
            for rem in grp:
                if rem != keep_idx:
                    test_rows.append({'kept_record': int(keep_idx), 'removed_record': int(rem), 'group_size': len(grp)})
        if test_rows:
            pd.DataFrame(test_rows).to_csv(dup_test_path, index=False, encoding='utf-8-sig')
            logging.info(f"發現重複，已輸出測試檔：{dup_test_path}")
        else:
            if os.path.exists(dup_test_path):
                try: os.remove(dup_test_path)
                except: pass

        logging.info(f"去重完成：精確移除 {removed_exact}，模糊移除 {len(to_remove)}，最終 {len(df_final)}")
        return df_final, manual_review

def perform_deduplication(df: pd.DataFrame):
    deduper = DeduplicationEngine()
    key_cols = ['company_name_clean','registered_country','reg_no_normalized']
    df_final, manual_review = deduper.find_duplicates(df, key_cols)
    dedup_path = os.path.join(Config.OUTPUT_PATH, Config.FILE_DEDUP)
    df_final.to_csv(dedup_path, index=False, encoding='utf-8-sig')
    with open(os.path.join(Config.OUTPUT_PATH, Config.FILE_MANUAL_REVIEW), 'w', encoding='utf-8') as f:
        json.dump(manual_review, f, ensure_ascii=False, indent=2)
    logging.info(f"去重結果已儲存：{dedup_path}")
    return df_final, manual_review

dedup_df, manual_review = perform_deduplication(assessed_df)

# ------------------------------
# STEP 7：報告生成（Excel、英文 PDF、驗證指南）
# ------------------------------
class ReportGenerator:
    """產生 Excel 與 PDF（PDF 為英文段落摘要）"""
    @staticmethod
    def summary_stats(df: pd.DataFrame) -> Dict:
        stats = {
            'total_companies': len(df),
            'real_companies': int(df['is_real_company'].sum()) if 'is_real_company' in df.columns else 0,
            'unique_companies': int(df['company_name_clean'].nunique()),
            'high_risk_count': int((df['risk_score'] >= 70).sum()),
            'lei_coverage': int(df['lei_enhanced_valid'].sum()) if 'lei_enhanced_valid' in df.columns else int(df['lei_enhanced'].notna().sum()),
            'ofac_hits': int(df['ofac_hit'].sum()) if 'ofac_hit' in df.columns else 0,
            'kyc_expired': int(df['kyc_refresh_due'].sum())
        }
        return stats

    @staticmethod
    def generate_excel(df: pd.DataFrame, outdir: str) -> str:
        out = os.path.join(outdir, Config.FILE_EXCEL)
        with pd.ExcelWriter(out, engine='openpyxl') as writer:
            cols = [c for c in ['record_id','company_name','company_name_clean','registered_country','reg_no','reg_no_normalized','risk_score','risk_category','lei_enhanced','lei_enhanced_valid','ofac_hit','ofac_confidence','kyc_refresh_due','days_until_next_review','aum_usd_millions','incorporation_date','last_kyc_review'] if c in df.columns]
            df[cols].to_excel(writer, sheet_name='Customer Risk Overview', index=False)
            high = df[df['risk_score'] >= 70]
            if not high.empty:
                high[cols].to_excel(writer, sheet_name='High Risk Customers', index=False)
            if 'ofac_hit' in df.columns:
                sanctions = df[df['ofac_hit'] == True]
                if not sanctions.empty:
                    scols = [c for c in ['record_id','company_name_clean','registered_country','ofac_hit','ofac_confidence','ofac_matched_entity','risk_score'] if c in df.columns]
                    sanctions[scols].to_excel(writer, sheet_name='Sanctions Hits', index=False)
            stats = ReportGenerator.summary_stats(df)
            stats_df = pd.DataFrame([{'Metric':'Total Companies','Value':stats['total_companies']},
                                     {'Metric':'LEI Coverage','Value':stats['lei_coverage']},
                                     {'Metric':'Sanctions Hits','Value':stats['ofac_hits']},
                                     {'Metric':'KYC Overdue','Value':stats['kyc_expired']},
                                     {'Metric':'Generated At','Value':datetime.now().strftime('%Y-%m-%d %H:%M:%S')}])
            stats_df.to_excel(writer, sheet_name='Summary', index=False)
        logging.info(f"Excel 已生成：{out}")
        return out

    @staticmethod
    def generate_pdf(df: pd.DataFrame, outdir: str) -> str:
        """
        生成英文報告（只有文字段落，不含表格）。
        標題使用正式銀行格式： "KYC Compliance Summary Report"
        """
        out = os.path.join(outdir, Config.FILE_PDF)
        doc = SimpleDocTemplate(out, pagesize=letter)
        styles = getSampleStyleSheet()
        story = []

        # 以內建英文字型（Helvetica 系列）產生樣式
        title_style = ParagraphStyle(name='TitleEN', parent=styles['Title'], fontName='Helvetica-Bold', fontSize=18)
        normal_style = ParagraphStyle(name='NormalEN', parent=styles['Normal'], fontName='Helvetica', fontSize=10, leading=14)

        # 英文標題（正式銀行格式）
        title = Paragraph("KYC Compliance Summary Report", title_style)
        story.append(title)
        story.append(Spacer(1, 12))

        # 英文段落摘要（簡潔、適合給管理層）
        stats = ReportGenerator.summary_stats(df)
        p1 = f"This report provides a concise summary of the KYC dataset processed by the v16 B version KYC compliance system."
        p2 = f"Total companies processed: {stats['total_companies']}. LEI coverage (validated): {stats['lei_coverage']}. Sanctions hits detected: {stats['ofac_hits']}."
        p3 = f"KYC overdue cases: {stats['kyc_expired']}. High-risk customers (risk score >=70): {stats['high_risk_count']}."
        p4 = "Notes: LEI enrichment attempted via GLEIF queries for all candidate names; OFAC screening performed against available SDN data (local copy or online source). Duplicate detection used MinHash LSH with dynamic weights influenced by risk profiles."

        story.append(Paragraph(p1, normal_style)); story.append(Spacer(1,8))
        story.append(Paragraph(p2, normal_style)); story.append(Spacer(1,8))
        story.append(Paragraph(p3, normal_style)); story.append(Spacer(1,8))
        story.append(Paragraph(p4, normal_style)); story.append(Spacer(1,12))

        ts = f"Report generated at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
        story.append(Paragraph(ts, normal_style))

        doc.build(story)
        logging.info(f"PDF 已生成：{out}")
        return out

    @staticmethod
    def generate_verification_guide(outdir: str) -> str:
        out = os.path.join(outdir, Config.FILE_GUIDE)
        with open(out, 'w', encoding='utf-8') as f:
            f.write("KYC System v16 Verification Guide\n")
            f.write(f"Generated at: {datetime.now().strftime('%Y-%m-%d %H:%M')}\n\n")
            f.write("Check the following items:\n")
            f.write("- LEI enrichment coverage and validity\n")
            f.write("- OFAC/SDN parsing and matching\n")
            f.write("- Deduplication output and manual review notes\n")
            f.write("- Risk distribution summary\n")
        logging.info(f"Verification guide saved: {out}")
        return out

# 產生報表
excel_path = ReportGenerator.generate_excel(dedup_df, Config.OUTPUT_PATH)
pdf_path = ReportGenerator.generate_pdf(dedup_df, Config.OUTPUT_PATH)
guide_path = ReportGenerator.generate_verification_guide(Config.OUTPUT_PATH)

# ------------------------------
# STEP 8：輸出檔案列舉與系統驗證摘要
# ------------------------------
def list_output_files():
    print("\n已輸出檔案位於：", Config.OUTPUT_PATH)
    for fn in sorted(os.listdir(Config.OUTPUT_PATH)):
        path = os.path.join(Config.OUTPUT_PATH, fn)
        try:
            size_kb = os.path.getsize(path) / 1024
            print(f" - {fn} ({size_kb:.1f} KB)")
        except:
            print(f" - {fn}")

list_output_files()

def system_validation(raw_df, final_df):
    print("\n系統驗證摘要：")
    total = len(final_df)
    missing_names = final_df['company_name_clean'].isna().sum()
    print(f" - 處理記錄數：{total}")
    print(f" - 缺失公司名：{missing_names}")
    lei_total = final_df['lei_enhanced'].notna().sum() if 'lei_enhanced' in final_df.columns else 0
    lei_valid = final_df['lei_enhanced_valid'].sum() if 'lei_enhanced_valid' in final_df.columns else 0
    lei_pct = (lei_valid / total * 100) if total > 0 else 0
    print(f" - LEI 覆蓋（驗證通過）：{lei_valid}/{lei_total} ({lei_pct:.2f}%)")
    ofac_hits = final_df['ofac_hit'].sum() if 'ofac_hit' in final_df.columns else 0
    print(f" - OFAC 命中：{ofac_hits} ({ofac_hits/total*100:.3f}%)")
    kyc_expired = final_df['kyc_refresh_due'].sum()
    print(f" - KYC 過期：{kyc_expired} ({kyc_expired/total*100:.1f}%)")
    dup_test_file = os.path.join(Config.OUTPUT_PATH, Config.FILE_DUP_TEST)
    print(f" - 去重組數量（測試環境清單）：{dup_test_file if os.path.exists(dup_test_file) else '無'}")

system_validation(raw_df, dedup_df)

print("\n完成：KYC 系統執行結束。")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.1/96.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.5/360.5 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 86.2 MB/s eta 0:00:00
Mounted at /content/drive


GLEIF 查詢:   0%|          | 0/12688 [00:00<?, ?it/s]

OFAC 篩查:   0%|          | 0/15670 [00:00<?, ?it/s]

風險評估:   0%|          | 0/15670 [00:00<?, ?it/s]

查找相似記錄:   0%|          | 0/15670 [00:00<?, ?it/s]


已輸出檔案位於： /content/drive/MyDrive/Colab Notebooks/KYC
 - KYC_Compliance_Summary_Report.pdf (2.1 KB)
 - KYC_project.ipynb (89.9 KB)
 - KYC合規報告_詳細版.xlsx (2209.3 KB)
 - Untitled0.ipynb (77.8 KB)
 - kyc_cleaned_data.csv (3685.0 KB)
 - kyc_deduplicated_data.csv (5896.4 KB)
 - kyc_lei_enhanced_data.csv (3826.6 KB)
 - kyc_raw_data.csv (1790.4 KB)
 - manual_review_notes.json (0.0 KB)
 - sdn.csv (5249.6 KB)
 - test_environment_duplicates.csv (30.7 KB)
 - verification_guide.txt (0.2 KB)

系統驗證摘要：
 - 處理記錄數：13928
 - 缺失公司名：0
 - LEI 覆蓋（驗證通過）：1201/1201 (8.62%)
 - OFAC 命中：87 (0.625%)
 - KYC 過期：1460 (10.5%)
 - 去重組數量（測試環境清單）：/content/drive/MyDrive/Colab Notebooks/KYC/test_environment_duplicates.csv

完成：KYC 系統執行結束。
